## Dataset & DataLoader 살표보기
- Pytorch에서 배치 크기만큼 데이터를 조절하기 위한 메카니즘
- Dataset : 사용 데이터를 기반으로 사용자 정의 클래스 작성
- DataLoader : 지정된 Dataset에서 지정된 batch size만큼 피쳐와 타겟을 추출하여 전달

### [1] 모듈 로딩 및 데이터 준비 <hr>

In [94]:
### 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

import torchvision

print(torchvision.__version__)

0.15.2a0


### [2] 데이터셋 생성 <hr>

#### [2-2] 사용자정의 데이터셋 생성

In [95]:
### 데이터 준비
filename = '../data/text/iris.csv'

irisDF = pd.read_csv(filename)
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [96]:
### [2-1] 사용자정의 Dataset 클래스
### - 데이터의 Tensor 변환 필요
from torch.utils.data import Dataset, DataLoader, TensorDataset

class DLDataset(Dataset):  # DLDataset : 자식    /    Dataset : 부모
    
    # 초기화 함수 콜백함수(callback functon)
    def __init__(self, x_data, y_data):
        super().__init__()
        
        # x,y 데이터 ==> ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        # ndarray ==> tensor
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)
        print('[target & target SHAPE]', self.target.shape, self.target.ndim)
        
    # 데이터셋의 갯수 체크 함수 콜백함수(callback functon)
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터+라벨 반환 콜백함수(callback functon)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

= [2-2] 데이터셋 인스턴스 생성

In [97]:
### 피쳐와 라벨로 분리
featureDF = irisDF.iloc[:,:-1]
targetDF = irisDF.species

print(f'{featureDF.shape}, {featureDF.ndim}D')
print(f'{targetDF.shape}, {targetDF.ndim}D')


(150, 4), 2D
(150,), 1D


In [98]:
# object 타입 타겟 ==> int 타입 타겟 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetDF)
targetNP = targetNP.reshape(-1,1)

targetNP.shape, targetNP.ndim


((150, 1), 2)

In [99]:
# ===> 데이터셋 생성 -> DF, NP
my_dataset = DLDataset(featureDF,targetNP)
my_dataset[0], featureDF.iloc[0], targetDF[0]

[target & target SHAPE] torch.Size([150, 1]) 2


((tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0])),
 sepal_length    5.1
 sepal_width     3.5
 petal_length    1.4
 petal_width     0.2
 Name: 0, dtype: float64,
 'setosa')

#### [2-3]  학습용, 검증용, 테스트용 Dataset <hr>

In [100]:
### ===> 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트 데이터 비율
seed = torch.Generator().manual_seed(42)
trainDS, validDS, testDS = random_split(my_dataset, lengths=[0.7,0.1,0.2], generator=seed) 
trainDS

In [101]:
print(f'trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, testDS => {len(testDS)}개')

trainDS => 105개, validDS => 15개, testDS => 30개


In [102]:
print(f'Subset 속성 =>\nindices : {trainDS.indices} \n dataset : {trainDS.dataset}')
print(f'Subset 속성 =>\nindices : {validDS.indices} \n dataset : {validDS.dataset}')
print(f'Subset 속성 =>\nindices : {testDS.indices} \n dataset : {testDS.dataset}')


Subset 속성 =>
indices : [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12] 
 dataset : <__main__.DLDataset object at 0x28a787580>
Subset 속성 =>
indices : [22, 104, 81, 1, 103, 125, 85, 2, 96, 128, 27, 118, 77, 110, 146] 
 dataset : <__main__.DLDataset object at 0x28a787580>
Subset 속성 =>
indices : [72, 139, 131, 60, 65, 92, 135, 83, 14, 34, 137, 10, 119, 9, 148, 79, 78, 70, 144, 143, 123, 115, 61, 132, 90, 46, 126, 136, 33, 138] 
 dataset : <__main__.DLDataset object at 0x28a787580>


[3] DataLoader 생성 : 학습용, 검증용, 테스트용

In [103]:
# DataLoader 생성
# drop_list 매개변수 : 배치 사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본 : False]
batch = 5
trainDL = DataLoader(trainDS, batch_size=batch)
validDL = DataLoader(validDS, batch_size=batch)
testDL = DataLoader(trainDS, batch_size=batch)

len(trainDS), len(validDS), len(testDS)


(105, 15, 30)

In [104]:
# Epoch당 반복 단위
print('batch_size : 10')
print(f'trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, testDF => {len(testDS)}개')
print(f'trainDL => {len(trainDL)}개, validDS => {len(validDL)}개, testDF => {len(testDL)}개')

batch_size : 10
trainDS => 105개, validDS => 15개, testDF => 30개
trainDL => 21개, validDS => 3개, testDF => 21개


In [105]:
# DataLoader 속성
for _ , (feature,target) in enumerate(validDL):
    print(f'[{_}] feature {feature.shape}')

[0] feature torch.Size([5, 4])
[1] feature torch.Size([5, 4])
[2] feature torch.Size([5, 4])


### [4] Model 클래스의 정의 : 입/출력 피쳐  수, 층 수, 은닉층의 노드/퍼셉트론/유닛 수 <hr>
- 구조 설계
    - 입력 층 : 입력(=피쳐 갯수) => iris 4개
    - 은닉 층 : 마음대로 알아서 잘
    - 출력 층 : 출력(=[분류] 타겟 클래스 갯수, [회귀] 1개) - 지금은 iris 분류

In [106]:
## 모델 클래스 정의
# 클래스명 : CModel

class CModel(nn.Module):
    # 구성요소 정의 함수
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.input_layer = nn.Linear(in_dim, 100)
        self.relu = nn.ReLU()
        self.hidden_layer = nn.Linear(100, 27)
        self.output_layer = nn.Linear(27, out_dim)
        
    # 순방향 학습 진행 함수
    def forward(self, x):
        x = self.input_layer(x)     # W1X1 + W2X2 + ... + WnXn + b 100개를 반환 함 
        x = self.relu(x)            # relu() 결과 100개 반환
        x = self.hidden_layer(x)    # W1X1 + W2X2 + ... + WnXn + b  27개를 반환 함
        x = self.relu(x)            # relu() 결과  27개 반환
        x = self.output_layer(x)    # W1X1 + W2X2 + ... + WnXn + b   3개를 반환 함 
        return x
    
    
class LinearClassifier(nn.Module):
    def __init__(self,in_,out_):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_,50),
            nn.ReLU(),
            nn.Linear(50,10),
            nn.ReLU(),
            nn.Linear(10,out_))
        
    def forward(self, x):
        x = self.layers(x)
        return x

In [107]:
targetDF.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [108]:
in_ = len(trainDS)
out_ = len(targetDF.unique())
in_, out_

(105, 3)

In [109]:
model2 = LinearClassifier(in_,out_)
model2

LinearClassifier(
  (layers): Sequential(
    (0): Linear(in_features=105, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=3, bias=True)
  )
)

### [5] 학습 준비 : 실행디바이스, 모델, 최적화, 손실함수, 학습횟수, 학습함수, 평가함수, 예측함수 <hr>

In [110]:
# 상수가 없기 때문에 대문자로 가독성 좋게 구분
# 상수로 만들기 위해서 튜플로 만들어서 쓰는 방법도 - (A) or A,

# 실행 디바이스 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습횟수
EPOCHS = 50


In [111]:
# 모델 인스턴스
IN, OUT = my_dataset.feature.shape[1], len(targetDF.unique())
model = CModel(IN, OUT).to(DEVICE)
print(f' IN : {IN}, OUT : {OUT}')
print(model)

 IN : 4, OUT : 3
CModel(
  (input_layer): Linear(in_features=4, out_features=100, bias=True)
  (relu): ReLU()
  (hidden_layer): Linear(in_features=100, out_features=27, bias=True)
  (output_layer): Linear(in_features=27, out_features=3, bias=True)
)


In [112]:
# 손실함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(model.parameters())



#### - 학습 및 검증 관련 함수 정의

In [123]:
### ==> 학습 진행함수
def traing():
    # 학습모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
    model.train()
    
    # 배치크기 만큼 학습 진행
    train_loss = []
    for cnt, (feature, target) in enumerate(trainDL):
        # print(cnt,feature,target)
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        target = target.squeeze()
        
        # 학습
        # print(f'feature => {feature}')
        # print()
        
        pre_target = model(feature)
        
        # 손실계산
        loss = LOSS_FN(pre_target, target)
        train_loss.append(loss)
        
        # W, b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()
    
        # 배치 단위 학습 진행 메시지 출력
        # print(f'[Train {cnt} loss] => {loss}')
    
    # 에포크 단위 학습 진행 메시지 출력
    # print(f'[Train loss] => {loss}')
    
    return train_loss
        

In [124]:
### ==> 검증 및 평가 진행함수
def testing():
    pass

In [125]:
### ==> 예측함수
def predict():
    pass

### [6] 학습 진행 <hr>

In [129]:
for eps in range(EPOCHS):
    # 학습
    train_loss = traing()
    
    # 검증
    testing()
    
    print(f'[{eps}/{EPOCHS}] {sum(train_loss)/len(train_loss)}')

[0/50] 1.2210878133773804
[1/50] 1.2210878133773804
[2/50] 1.2210878133773804
[3/50] 1.2210878133773804
[4/50] 1.2210878133773804
[5/50] 1.2210878133773804
[6/50] 1.2210878133773804
[7/50] 1.2210878133773804
[8/50] 1.2210878133773804
[9/50] 1.2210878133773804
[10/50] 1.2210878133773804
[11/50] 1.2210878133773804
[12/50] 1.2210878133773804
[13/50] 1.2210878133773804
[14/50] 1.2210878133773804
[15/50] 1.2210878133773804
[16/50] 1.2210878133773804
[17/50] 1.2210878133773804
[18/50] 1.2210878133773804
[19/50] 1.2210878133773804
[20/50] 1.2210878133773804
[21/50] 1.2210878133773804
[22/50] 1.2210878133773804
[23/50] 1.2210878133773804
[24/50] 1.2210878133773804
[25/50] 1.2210878133773804
[26/50] 1.2210878133773804
[27/50] 1.2210878133773804
[28/50] 1.2210878133773804
[29/50] 1.2210878133773804
[30/50] 1.2210878133773804
[31/50] 1.2210878133773804
[32/50] 1.2210878133773804
[33/50] 1.2210878133773804
[34/50] 1.2210878133773804
[35/50] 1.2210878133773804
[36/50] 1.2210878133773804
[37/50] 1.2